# Importing the data from kaggle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hotsonhonet","key":"e6cb444362df8bdc68a95a96af34b13c"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hotsonhonet/the-great-indian-hiring-challenge

 41% 9.00M/21.8M [00:00<00:00, 33.0MB/s]
100% 21.8M/21.8M [00:00<00:00, 55.0MB/s]


In [ ]:
!ls

kaggle.json  sample_data  the-great-indian-hiring-challenge.zip


In [ ]:
from zipfile import ZipFile

with ZipFile("the-great-indian-hiring-challenge.zip", "r") as file:
    file.extractall()
file.close()

In [ ]:
!ls Participants_Data_TGIH

Participants_Data_TGIH


In [ ]:
!pwd

/content


In [18]:
train_loc = "/content/Participants_Data_TGIH/Participants_Data_TGIH/Train.csv"
test_loc = "/content/Participants_Data_TGIH/Participants_Data_TGIH/Test.csv"

eda_tntt = "/content/tntt_features_fromEDA_2.csv"

# Importing Modules

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import chi2, f_regression, mutual_info_regression, SelectKBest, SelectPercentile, RFE
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from xgboost import XGBRegressor



# Loading our Data

In [19]:
tntt_features = pd.read_csv(eda_tntt)
train_data = pd.read_csv(train_loc)
test_data = pd.read_csv(test_loc)

In [20]:
train_size = train_data.shape[0]
test_size = test_data.shape[0]

In [23]:
tntt_features.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [24]:
train_df = tntt_features.iloc[:train_size, :]
test_df = tntt_features.iloc[train_size:, :]

In [27]:
train_df = pd.concat([train_df, train_data[["UnitPrice"]]], axis = 1)

In [29]:
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,Date,Time,year,weekofyear,month,dayofweek,weekend,cus_InvoiceNo_group,cus_StockCode_group,cus_Description_group,cus_Quantity_group,cus_InvoiceDate_group,cus_Country_group,cus_Date_group,cus_Time_group,cus_year_group,cus_weekofyear_group,cus_month_group,cus_dayofweek_group,cus_weekend_group,Quatity_type,Max_InvoiceNo,Max_StockCode,Max_Description,Max_Quantity,Max_InvoiceDate,Max_CustomerID,Max_Country,Max_Date,Max_Time,Max_year,Max_weekofyear,Max_month,Max_dayofweek,Max_weekend,Max_cus_InvoiceNo_group,Max_cus_StockCode_group,Max_cus_Description_group,Max_cus_Quantity_group,Max_cus_InvoiceDate_group,Max_cus_Country_group,Max_cus_Date_group,Max_cus_Time_group,Max_cus_year_group,Max_cus_weekofyear_group,Max_cus_month_group,Max_cus_dayofweek_group,Max_cus_weekend_group,Max_Quatity_type,CmnCustomer,CmnStockID,Freq_InvoiceNo,Freq_StockCode,Freq_Description,Freq_Quantity,Freq_InvoiceDate,Freq_CustomerID,Freq_Country,UnitPrice
0,6141,1583,144,3,2011-05-06 16:54:00,14056.0,35,2011-05-06,16:54:00,2011,18,5,4,0,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,3,70,26948,0,0,2069,3.75
1,6349,1300,3682,6,2011-05-11 07:35:00,13098.0,35,2011-05-11,07:35:00,2011,19,5,2,0,605,605,605,605,605,605,605,605,605,605,605,605,605,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,64,64,37688,0,0,758,1.95
2,16783,2178,1939,4,2011-11-20 13:20:00,15044.0,35,2011-11-20,13:20:00,2011,46,11,6,1,461,461,461,461,461,461,461,461,461,461,461,461,461,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,5,1,32183,0,0,32,5.95
3,16971,2115,2983,1,2011-11-22 12:07:00,15525.0,35,2011-11-22,12:07:00,2011,47,11,1,0,180,180,180,180,180,180,180,180,180,180,180,180,180,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401,0.83
4,6080,1210,2886,12,2011-05-06 09:00:00,13952.0,35,2011-05-06,09:00:00,2011,18,5,4,0,138,138,138,138,138,138,138,138,138,138,138,138,138,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,329,250,60033,0,0,695,1.65


In [30]:
test_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,Date,Time,year,weekofyear,month,dayofweek,weekend,cus_InvoiceNo_group,cus_StockCode_group,cus_Description_group,cus_Quantity_group,cus_InvoiceDate_group,cus_Country_group,cus_Date_group,cus_Time_group,cus_year_group,cus_weekofyear_group,cus_month_group,cus_dayofweek_group,cus_weekend_group,Quatity_type,Max_InvoiceNo,Max_StockCode,Max_Description,Max_Quantity,Max_InvoiceDate,Max_CustomerID,Max_Country,Max_Date,Max_Time,Max_year,Max_weekofyear,Max_month,Max_dayofweek,Max_weekend,Max_cus_InvoiceNo_group,Max_cus_StockCode_group,Max_cus_Description_group,Max_cus_Quantity_group,Max_cus_InvoiceDate_group,Max_cus_Country_group,Max_cus_Date_group,Max_cus_Time_group,Max_cus_year_group,Max_cus_weekofyear_group,Max_cus_month_group,Max_cus_dayofweek_group,Max_cus_weekend_group,Max_Quatity_type,CmnCustomer,CmnStockID,Freq_InvoiceNo,Freq_StockCode,Freq_Description,Freq_Quantity,Freq_InvoiceDate,Freq_CustomerID,Freq_Country
284780,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35,2011-02-22,15:22:00,2011,8,2,1,0,502,502,502,502,502,502,502,502,502,502,502,502,502,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,3,70,26948,0,0,2069
284781,617,510,3457,1,2010-12-08 12:46:00,17341.0,35,2010-12-08,12:46:00,2010,49,12,2,0,183,183,183,183,183,183,183,183,183,183,183,183,183,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401
284782,14653,604,694,36,2011-10-25 13:53:00,15158.0,35,2011-10-25,13:53:00,2011,43,10,1,0,7,7,7,7,7,7,7,7,7,7,7,7,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28,16,20,3751,0,0,244
284783,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35,2011-06-27,12:38:00,2011,26,6,0,0,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,30,140,58003,0,0,17
284784,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35,2011-11-06,16:14:00,2011,44,11,6,1,233,233,233,233,233,233,233,233,233,233,233,233,233,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406824,7813,2487,1289,12,2011-06-09 09:44:00,15214.0,35,2011-06-09,09:44:00,2011,23,6,3,0,110,110,110,110,110,110,110,110,110,110,110,110,110,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,329,250,60033,0,0,695
406825,8694,3069,217,16,2011-06-28 15:47:00,16200.0,35,2011-06-28,15:47:00,2011,26,6,1,0,179,179,179,179,179,179,179,179,179,179,179,179,179,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,63,10,3946,0,0,182
406826,915,1419,3474,6,2010-12-13 12:26:00,15555.0,35,2010-12-13,12:26:00,2010,50,12,0,0,925,925,925,925,925,925,925,925,925,925,925,925,925,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,64,64,37688,0,0,758
406827,8156,1631,1434,1,2011-06-16 12:45:00,15907.0,35,2011-06-16,12:45:00,2011,24,6,3,0,249,249,249,249,249,249,249,249,249,249,249,249,249,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401


In [32]:
train_cols = list(train_df.columns)
features_label = train_df.drop(['InvoiceDate', 'Date', 'Time'], axis = 1)
features_label

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,year,weekofyear,month,dayofweek,weekend,cus_InvoiceNo_group,cus_StockCode_group,cus_Description_group,cus_Quantity_group,cus_InvoiceDate_group,cus_Country_group,cus_Date_group,cus_Time_group,cus_year_group,cus_weekofyear_group,cus_month_group,cus_dayofweek_group,cus_weekend_group,Quatity_type,Max_InvoiceNo,Max_StockCode,Max_Description,Max_Quantity,Max_InvoiceDate,Max_CustomerID,Max_Country,Max_Date,Max_Time,Max_year,Max_weekofyear,Max_month,Max_dayofweek,Max_weekend,Max_cus_InvoiceNo_group,Max_cus_StockCode_group,Max_cus_Description_group,Max_cus_Quantity_group,Max_cus_InvoiceDate_group,Max_cus_Country_group,Max_cus_Date_group,Max_cus_Time_group,Max_cus_year_group,Max_cus_weekofyear_group,Max_cus_month_group,Max_cus_dayofweek_group,Max_cus_weekend_group,Max_Quatity_type,CmnCustomer,CmnStockID,Freq_InvoiceNo,Freq_StockCode,Freq_Description,Freq_Quantity,Freq_InvoiceDate,Freq_CustomerID,Freq_Country,UnitPrice
0,6141,1583,144,3,14056.0,35,2011,18,5,4,0,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1128,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,3,70,26948,0,0,2069,3.75
1,6349,1300,3682,6,13098.0,35,2011,19,5,2,0,605,605,605,605,605,605,605,605,605,605,605,605,605,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,64,64,37688,0,0,758,1.95
2,16783,2178,1939,4,15044.0,35,2011,46,11,6,1,461,461,461,461,461,461,461,461,461,461,461,461,461,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,5,1,32183,0,0,32,5.95
3,16971,2115,2983,1,15525.0,35,2011,47,11,1,0,180,180,180,180,180,180,180,180,180,180,180,180,180,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401,0.83
4,6080,1210,2886,12,13952.0,35,2011,18,5,4,0,138,138,138,138,138,138,138,138,138,138,138,138,138,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,329,250,60033,0,0,695,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284775,3408,1706,1081,6,17965.0,35,2011,9,3,1,0,329,329,329,329,329,329,329,329,329,329,329,329,329,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,64,64,37688,0,0,758,1.65
284776,13982,2184,2947,16,16793.0,35,2011,41,10,3,0,16,16,16,16,16,16,16,16,16,16,16,16,16,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,63,10,3946,0,0,182,0.65
284777,6641,667,3632,4,14755.0,35,2011,20,5,0,0,237,237,237,237,237,237,237,237,237,237,237,237,237,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,5,1,32183,0,0,32,8.50
284778,13820,1576,3089,48,14866.0,35,2011,41,10,1,0,103,103,103,103,103,103,103,103,103,103,103,103,103,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,6,6,309,5988,0,0,0,3.45


# Models

In [40]:
x_train, x_test, y_train, y_test = train_test_split(features_label.iloc[:, :-1], features_label.iloc[:, -1], test_size = 30, random_state = 30)

In [41]:
dtr = DecisionTreeRegressor(criterion='mse')
adaboost = AdaBoostRegressor(base_estimator=dtr)

adaboost.fit(x_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                       criterion='mse',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                                                       random_state=None,
                                                       splitter='best'),
               

In [43]:
preds = adaboost.predict(x_test)
score = mse(preds, y_test)
score = score**0.5
score

1.810666543937158

In [45]:
t_features = test_df.drop(['InvoiceDate', 'Date', 'Time'], axis = 1)
t_features

,InvoiceNo,StockCode,Description,Quantity,CustomerID,Country,year,weekofyear,month,dayofweek,weekend,cus_InvoiceNo_group,cus_StockCode_group,cus_Description_group,cus_Quantity_group,cus_InvoiceDate_group,cus_Country_group,cus_Date_group,cus_Time_group,cus_year_group,cus_weekofyear_group,cus_month_group,cus_dayofweek_group,cus_weekend_group,Quatity_type,Max_InvoiceNo,Max_StockCode,Max_Description,Max_Quantity,Max_InvoiceDate,Max_CustomerID,Max_Country,Max_Date,Max_Time,Max_year,Max_weekofyear,Max_month,Max_dayofweek,Max_weekend,Max_cus_InvoiceNo_group,Max_cus_StockCode_group,Max_cus_Description_group,Max_cus_Quantity_group,Max_cus_InvoiceDate_group,Max_cus_Country_group,Max_cus_Date_group,Max_cus_Time_group,Max_cus_year_group,Max_cus_weekofyear_group,Max_cus_month_group,Max_cus_dayofweek_group,Max_cus_weekend_group,Max_Quatity_type,CmnCustomer,CmnStockID,Freq_InvoiceNo,Freq_StockCode,Freq_Description,Freq_Quantity,Freq_InvoiceDate,Freq_CustomerID,Freq_Country
284780,3139,1709,1079,3,16161.0,35,2011,8,2,1,0,502,502,502,502,502,502,502,502,502,502,502,502,502,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,3,70,26948,0,0,2069
284781,617,510,3457,1,17341.0,35,2010,49,12,2,0,183,183,183,183,183,183,183,183,183,183,183,183,183,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401
284782,14653,604,694,36,15158.0,35,2011,43,10,1,0,7,7,7,7,7,7,7,7,7,7,7,7,7,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,28,16,20,3751,0,0,244
284783,8634,1478,3473,2,16033.0,35,2011,26,6,0,0,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1152,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,30,140,58003,0,0,17
284784,15546,3216,871,1,15351.0,35,2011,44,11,6,1,233,233,233,233,233,233,233,233,233,233,233,233,233,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406824,7813,2487,1289,12,15214.0,35,2011,23,6,3,0,110,110,110,110,110,110,110,110,110,110,110,110,110,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,329,250,60033,0,0,695
406825,8694,3069,217,16,16200.0,35,2011,26,6,1,0,179,179,179,179,179,179,179,179,179,179,179,179,179,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12,63,10,3946,0,0,182
406826,915,1419,3474,6,15555.0,35,2010,50,12,0,0,925,925,925,925,925,925,925,925,925,925,925,925,925,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,64,64,37688,0,0,758
406827,8156,1631,1434,1,15907.0,35,2011,24,6,3,0,249,249,249,249,249,249,249,249,249,249,249,249,249,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,21,110,73314,0,0,401


In [51]:
t_preds = adaboost.predict(t_features)
t_preds

array([1.65, 1.25, 2.1 , ..., 1.65, 3.95, 4.15])

In [54]:
t_preds = pd.DataFrame(t_preds, columns=["UnitPrice"])
t_preds.reset_index()
t_preds.to_csv("Submission.csv", index = False)